<a href="https://colab.research.google.com/github/prorab-bit/danila/blob/main/streebog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
pip install aiogram nest_asyncio

In [15]:
import asyncio
import logging
import nest_asyncio
from aiogram import Bot, Dispatcher, types
from aiogram.filters import Command
from typing import Literal

nest_asyncio.apply()

logging.basicConfig(level=logging.INFO)

TOKEN = "7388487689:AAFYJPKGPu704iwnnwgaxDlnWD9fn3ym9ss"
bot = Bot(token=TOKEN)
dp = Dispatcher()

# -----------------------------------------------------
# Реализация Стрибог (GOST R 34.11-2012)
# -----------------------------------------------------
BLOCK_SIZE = 64

IV512 = bytes([1] * BLOCK_SIZE)
IV256 = bytes([0] * BLOCK_SIZE)

SBOX = [
    252, 238, 221, 17, 207, 110, 49, 22, 251, 196, 250, 218, 35, 197, 4, 77,
    233, 119, 240, 219, 147, 46, 153, 186, 23, 54, 241, 187, 20, 205, 95, 193,
    249, 24, 101, 90, 226, 92, 239, 33, 129, 28, 60, 66, 139, 1, 142, 79,
    5, 132, 2, 174, 227, 106, 143, 160, 6, 11, 237, 152, 127, 212, 211, 31,
    235, 52, 44, 81, 234, 200, 72, 171, 242, 42, 104, 162, 253, 58, 206, 204,
    181, 112, 14, 86, 8, 12, 118, 18, 191, 114, 19, 71, 156, 183, 93, 135,
    21, 161, 150, 41, 16, 123, 154, 199, 243, 145, 120, 111, 157, 158, 178, 177,
    50, 117, 25, 61, 255, 53, 138, 126, 109, 84, 198, 128, 195, 189, 13, 87,
    223, 245, 36, 169, 62, 168, 67, 201, 215, 121, 214, 246, 124, 34, 185, 3,
    224, 15, 236, 222, 122, 148, 176, 188, 220, 232, 40, 80, 78, 51, 10, 74,
    167, 151, 96, 115, 30, 0, 98, 68, 26, 184, 56, 130, 100, 159, 38, 65,
    173, 69, 70, 146, 39, 94, 85, 47, 140, 163, 165, 125, 105, 213, 149, 59,
    7, 88, 179, 64, 134, 172, 29, 247, 48, 55, 107, 228, 136, 217, 231, 137,
    225, 27, 131, 73, 76, 63, 248, 254, 141, 83, 170, 144, 202, 216, 133, 97,
    32, 113, 103, 164, 45, 43, 9, 91, 203, 155, 37, 208, 190, 229, 108, 82,
    89, 166, 116, 210, 230, 244, 180, 192, 209, 102, 175, 194, 57, 75, 99, 182
]

PBOX = [
    0, 8, 16, 24, 32, 40, 48, 56,
    1, 9, 17, 25, 33, 41, 49, 57,
    2, 10, 18, 26, 34, 42, 50, 58,
    3, 11, 19, 27, 35, 43, 51, 59,
    4, 12, 20, 28, 36, 44, 52, 60,
    5, 13, 21, 29, 37, 45, 53, 61,
    6, 14, 22, 30, 38, 46, 54, 62,
    7, 15, 23, 31, 39, 47, 55, 63
]

A = [
    0x8e, 0x20, 0xfa, 0xa7, 0x2b, 0xa0, 0xb4, 0x70,
    0x47, 0x10, 0x7d, 0xdd, 0x9b, 0x50, 0x5a, 0x38,
    0xad, 0x08, 0xb0, 0xe0, 0xc3, 0x28, 0x2d, 0x1c,
    0xd8, 0x04, 0x58, 0x70, 0xef, 0x14, 0x98, 0x0e,
    0x6c, 0x02, 0x2c, 0x38, 0xf9, 0x0a, 0x4c, 0x07,
    0x36, 0x01, 0x16, 0x1c, 0xf2, 0x05, 0x26, 0x8d,
    0x1b, 0x8e, 0x0b, 0x0e, 0x79, 0x8c, 0x13, 0xc8,
    0x83, 0x47, 0x8b, 0x07, 0xb2, 0x46, 0x87, 0x64
]

C = [bytes([i + 1] + [0] * 63) for i in range(12)]

POLY = 0xC3

class Streebog:
    def __init__(self, digest_size: Literal[32, 64] = 64):
        self.h = IV512 if digest_size == 64 else IV256
        self.n = bytes(BLOCK_SIZE)
        self.sigma = bytes(BLOCK_SIZE)
        self.digest_size = digest_size

    def _xor(self, a: bytes, b: bytes) -> bytes:
        return bytes(x ^ y for x, y in zip(a, b))

    def _S(self, data: bytes) -> bytes:
        return bytes(SBOX[b] for b in data)

    def _P(self, data: bytes) -> bytes:
        return bytes(data[PBOX[i]] for i in range(BLOCK_SIZE))

    def _L(self, data: bytes) -> bytes:
        result = bytearray(BLOCK_SIZE)
        for i in range(8):
            v = data[i * 8:(i + 1) * 8]
            r = 0
            for j in range(8):
                for k in range(8):
                    if v[j] & (1 << (7 - k)):
                        r ^= A[j * 8 + k]
            result[i * 8:(i + 1) * 8] = r.to_bytes(8, 'big')
        return bytes(result)

    def _LPS(self, data: bytes) -> bytes:
        return self._L(self._P(self._S(data)))

    def _E(self, K: bytes, m: bytes) -> bytes:
        state = self._xor(K, m)
        for i in range(12):
            state = self._LPS(state)
            K = self._LPS(self._xor(K, C[i]))
            state = self._xor(state, K)
        return state

    def _pad(self, data: bytes) -> bytes:
        return data + b'\x01' + b'\x00' * (BLOCK_SIZE - len(data) - 1)

    def update(self, message: bytes):
        for i in range(0, len(message), BLOCK_SIZE):
            block = message[i:i + BLOCK_SIZE]
            if len(block) < BLOCK_SIZE:
                block = self._pad(block)
            self.h = self._E(self.h, block)
            self.n = (int.from_bytes(self.n, "big") + (len(block) * 8)).to_bytes(BLOCK_SIZE, "big")
            self.sigma = (int.from_bytes(self.sigma, "big") + int.from_bytes(block, "big")).to_bytes(BLOCK_SIZE, "big")

    def digest(self) -> bytes:
        self.h = self._E(self.h, self.n)
        self.h = self._E(self.h, self.sigma)
        return self.h[::-1][:self.digest_size]

    def hexdigest(self) -> str:
        return self.digest().hex()

def streebog(message: bytes, digest_size: Literal[32, 64] = 64) -> str:
    g = Streebog(digest_size)
    g.update(message)
    return g.hexdigest()

# -----------------------------------------------------
# Telegram Bot: обработка сообщений
# -----------------------------------------------------

@dp.message(Command("start"))
async def start(message: types.Message):
    await message.answer(
        "Привет! Отправь текст, и я верну хеш по алгоритму Стрибог (GOST R 34.11-2012).\n\n"
        "Используй:\n"
        "`512 текст` — для Стрибог-512\n"
        "`256 текст` — для Стрибог-256\n"
    )

@dp.message(Command("help"))
async def help(message: types.Message):
    await message.answer(
        "Просто отправь сообщение, и я верну хеш. Примеры:\n"
        "`512 hello` — Стрибог-512\n"
        "`256 hello` — Стрибог-256\n"
    )

@dp.message()
async def handle_text(message: types.Message):
    try:
        if not message.text:
            return

        text = message.text.strip()

        if text.startswith("256 "):
            bits = 256
            text = text[4:]
        elif text.startswith("512 "):
            bits = 512
            text = text[4:]
        else:
            bits = 512

        msg = text.encode('utf-8')
        hash_value = streebog(msg, bits // 8)
        await message.answer(f"Хеш ({bits}-битный):\n{hash_value}")
    except Exception as e:
        await message.answer(f"Произошла ошибка: {str(e)}")

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(dp.start_polling(bot))
